# 1-03 Time Aware DataFrames

In [1]:
import pandas as pd
pd.set_option('display.width', 76)
pd.set_option('display.max_colwidth', 10)

## Time Aware Dataframes

* With any Time Series, there are two values of interest:
    - The time element (could also be a date)
    - A value for that time element
* In pandas, the reccomended way to deal with this is to convert the time element to an index
* This is called a DateTimeIndex
* We encountered this previously when using the date_range function
* Using a DateTimeIndex provides us with many advantages and generally makes things easier
* We can create a DateTimeIndex in two ways:
    - When we create the DataFrame
    - After we create the DataFrame

* We'll start by creating a basic dataframe and then adding a time_index:

In [18]:
df = pd.DataFrame({"Value":range(5)})
df.head(5)

,Value
0,0
1,1
2,2
3,3
4,4


In [19]:
my_dates = pd.date_range("01-01-2018", periods=5, freq="M")
df.index = my_dates
df

,Value
2018-01-31,0
2018-02-28,1
2018-03-31,2
2018-04-30,3
2018-05-31,4


* Note how the month defaults to month end. If we wanted the beginning of a month, we could use a Date Offset:

In [6]:
from pandas.tseries.offsets import MonthBegin

In [9]:
my_dates = pd.date_range("01-01-2018", periods=5, freq="M")
my_dates - MonthBegin()

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01'],
              dtype='datetime64[ns]', freq='M')

* We can also do this at the point we create the dataframe as follows:

In [11]:
df = pd.DataFrame(
    data={"Value":range(5)},
    index=my_dates - MonthBegin()
)
df

,Value
2018-01-01,0
2018-02-01,1
2018-03-01,2
2018-04-01,3
2018-05-01,4


* We could also use the set_index method

In [23]:
df = pd.DataFrame({"Value":range(5)})
df.set_index(my_dates, inplace=True)
df

,Value
2018-01-31,0
2018-02-28,1
2018-03-31,2
2018-04-30,3
2018-05-31,4


## Selecting Records

* We can use pandas standard indexing to select dates from the DateTimeIndex

In [24]:
# select rows by a specific value
df.loc["2018-01-31"]

Value    0
Name: 2018-01-31 00:00:00, dtype: int64

In [25]:
# select multiple rows using a range of values
df["2018-01-01":"2018-03-01"]

,Value
2018-01-31,0
2018-02-28,1


In [26]:
# select multiple rows using a hierarchically higher value
df["2018-01"]

,Value
2018-01-31,0


In [27]:
df["2018"]

,Value
2018-01-31,0
2018-02-28,1
2018-03-31,2
2018-04-30,3
2018-05-31,4


## Exercise

1. Load in dji.csv and ensure the Date column is set as the index.
2. Select all values from the summer (between May and August).
3. Is there anything unusual about the dates in this dataset?

#### 1. Load in dji.csv and ensure the Date column is set as the index.

In [31]:
dji = pd.read_csv(
    filepath_or_buffer='dji.csv',
    parse_dates=['Date'],
    index_col='Date'
)
dji.head(5)

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-12-31,17987....,18043....,17820....,17823....,828400...,17823....
2014-12-30,18035....,18035....,17959....,17983....,474900...,17983....
2014-12-29,18046....,18073....,18021....,18038....,538700...,18038....
2014-12-26,18038....,18103....,18038....,18053....,525700...,18053....
2014-12-24,18035....,18086....,18027....,18030....,428700...,18030....


#### 2. Select all values from the summer (between May and August).

In [35]:
dji['2014-08':'2014-05'].head()

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-08-29,17083....,17110....,17035....,17098....,815000...,17098....
2014-08-28,17119....,17119....,17018....,17079....,518600...,17079....
2014-08-27,17111....,17134....,17090....,17122....,616900...,17122....
2014-08-26,17079....,17153....,17079....,17106....,507100...,17106....
2014-08-25,17011....,17124....,17011....,17076....,574000...,17076....


#### 3. Is there anything unusual about the dates in this dataset?

They go backwards!

In [40]:
dji.sort_index(inplace=True)
dji['2014-05':'2014-08'].head()

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-05-01,16580....,16604....,16525....,16558....,756300...,16558....
2014-05-02,16562....,16620....,16488....,16512....,789100...,16512....
2014-05-05,16509....,16547....,16377....,16530....,703200...,16530....
2014-05-06,16529....,16529....,16399....,16401....,743400...,16401....
2014-05-07,16401....,16522....,16357....,16518....,938700...,16518....


## Resampling

* When analysing time series, it helps if the time series is consistent.
* It's far easier to analyse time-series with a fixed number of observations
* We can convert to a fixed number of observations using a process called resampling
* Pandas comes with some tools to make this easy

In [44]:
df = pd.DataFrame(
    data={"Value":[10, 20, 40, 80, 160, 320]},
    index=pd.date_range("01-01-2018", periods=6, freq="30T")
)
df

,Value
2018-01-01 00:00:00,10
2018-01-01 00:30:00,20
2018-01-01 01:00:00,40
2018-01-01 01:30:00,80
2018-01-01 02:00:00,160
2018-01-01 02:30:00,320


* As we can see this dataset has 30 minute intervals
* If we want to convert it to hourly we can use the resample method

In [47]:
hourly_df = df.resample("H").sum()
hourly_df

,Value
2018-01-01 00:00:00,30
2018-01-01 01:00:00,120
2018-01-01 02:00:00,480


* Note that we have to supply an aggregation method

In [54]:
hourly_df = df.resample("H").mean()
hourly_df

,Value
2018-01-01 00:00:00,15
2018-01-01 01:00:00,60
2018-01-01 02:00:00,240


* This is called downsampling because we have decreased the frequency of the samples
* We can also upsample to increase the frequency as follows:

In [55]:
hourly_df.resample("30T").asfreq()

,Value
2018-01-01 00:00:00,15.0
2018-01-01 00:30:00,NaN
2018-01-01 01:00:00,60.0
2018-01-01 01:30:00,NaN
2018-01-01 02:00:00,240.0


* However we're now left with empty values. We can input these with an infilling function such as bfill or ffill

In [56]:
# Forward fill
hourly_df.resample("30T").ffill()

,Value
2018-01-01 00:00:00,15
2018-01-01 00:30:00,15
2018-01-01 01:00:00,60
2018-01-01 01:30:00,60
2018-01-01 02:00:00,240


In [58]:
# Back fill
hourly_df.resample("30T").bfill()

,Value
2018-01-01 00:00:00,15
2018-01-01 00:30:00,60
2018-01-01 01:00:00,60
2018-01-01 01:30:00,240
2018-01-01 02:00:00,240


## Diff

* The diff method allows us to compare the difference between two values from the same column

In [61]:
sales_data = pd.DataFrame(
    data={"Sales":[10, 20, 40, 80]},
    index=pd.date_range("01-07-2018", periods=4, freq="M")
)
sales_data.head()

,Sales
2018-01-31,10
2018-02-28,20
2018-03-31,40
2018-04-30,80


* If we call diff, it will default to the previous row:

In [67]:
sales_data['Sales -1'] = sales_data['Sales'].diff()
sales_data

,Sales,Sales -1
2018-01-31,10,NaN
2018-02-28,20,10.0
2018-03-31,40,20.0
2018-04-30,80,40.0


* We can supply an argument to the diff method to specify different values to compare

In [79]:
sales_data['Sales -2'] = sales_data['Sales'].diff(periods=2)
sales_data

,Sales,Sales -1,Sales -2,Sales +2,previous_month
2018-01-31,10,NaN,NaN,-30.0,NaN
2018-02-28,20,10.0,NaN,-60.0,10.0
2018-03-31,40,20.0,30.0,NaN,20.0
2018-04-30,80,40.0,60.0,NaN,40.0


* We can look forward by supplying a negative value

In [74]:
sales_data['Sales +2'] = sales_data['Sales'].diff(periods=-2)
sales_data

,Sales,Sales -1,Sales -2,Sales +2
2018-01-31,10,NaN,NaN,-30.0
2018-02-28,20,10.0,NaN,-60.0
2018-03-31,40,20.0,30.0,NaN
2018-04-30,80,40.0,60.0,NaN


Some Tips:
* Diff isn't restricted to time series - you can use it for anything
* As such it doesn't take account of the time element
* You can change the behaviour between rows and columns by supplying an axis= keyword argument.
* It only calculates the difference between two values.

We can also combine 

## Shift

* Shift works similarly to diff, but instead it simply shifts the value instead of calculating the difference.

In [80]:
sales_data["previous_month"] = sales_data['Sales'].shift(periods=1)
sales_data

,Sales,Sales -1,Sales -2,Sales +2,previous_month
2018-01-31,10,NaN,NaN,-30.0,NaN
2018-02-28,20,10.0,NaN,-60.0,10.0
2018-03-31,40,20.0,30.0,NaN,20.0
2018-04-30,80,40.0,60.0,NaN,40.0


* You can supply a different period argument to shift the values accordingly
* We can also use shift to be more creative with our infilling when resampling

In [117]:
df_rs = hourly_df.resample("30T").asfreq()
df_rs

,Value
2018-01-01 00:00:00,15.0
2018-01-01 00:30:00,NaN
2018-01-01 01:00:00,60.0
2018-01-01 01:30:00,NaN
2018-01-01 02:00:00,240.0


In [118]:
df_rs['Value'] = df_rs['Value'].fillna(
    (df_rs['Value'].shift() + df_rs['Value'].shift(-1)) / 2
)
df_rs

,Value
2018-01-01 00:00:00,15.0
2018-01-01 00:30:00,37.5
2018-01-01 01:00:00,60.0
2018-01-01 01:30:00,150.0
2018-01-01 02:00:00,240.0


## Rolling Windows

* 'Rolling Windows' are where we aggregate data for a certain time period repeatedly.
* They're used for things like 'Rolling 12 month averages' which help balance out seasonal variations
* As you might expect, Pandas has some functionality for this through the rolling method

In [91]:
df = pd.DataFrame(
    data={"Value":[10, 20, 40, 80]*10},
    index=pd.date_range("01-01-2018", periods=40, freq="D")
)
df.head(10)

,Value
2018-01-01,10
2018-01-02,20
2018-01-03,40
2018-01-04,80
2018-01-05,10
2018-01-06,20
2018-01-07,40
2018-01-08,80
2018-01-09,10
2018-01-10,20


In [92]:
df.rolling(window=7).mean().head(10)

,Value
2018-01-01,NaN
2018-01-02,NaN
2018-01-03,NaN
2018-01-04,NaN
2018-01-05,NaN
2018-01-06,NaN
2018-01-07,31.428571
2018-01-08,41.428571
2018-01-09,40.000000
2018-01-10,37.142857


* The window= keyword argument allows you to select how many periods you want to include in the rolling window
* As with resample, you have to include an aggregation method.
* By default, the rolling method will only provide a value when it has at least the number of values you specified in the window= keyword argument
* We can change this with the min_periods= keyword argument

In [103]:
df.rolling(window=7, min_periods=6).mean().head(10)

,Value
2018-01-01,NaN
2018-01-02,NaN
2018-01-03,NaN
2018-01-04,NaN
2018-01-05,NaN
2018-01-06,30.000000
2018-01-07,31.428571
2018-01-08,41.428571
2018-01-09,40.000000
2018-01-10,37.142857


* We can also use window as a custom function in order to pass to apply:

In [99]:
df.rolling(window=7).apply(lambda x: x.max()-x.min(), raw=False)

,Value
2018-01-01,NaN
2018-01-02,NaN
2018-01-03,NaN
2018-01-04,NaN
2018-01-05,NaN
2018-01-06,NaN
2018-01-07,70.0
2018-01-08,70.0
2018-01-09,70.0
2018-01-10,70.0


* I don't make the training courses... This is a poor example!!
* What it's doing...
    - Calculating the min and max 'Value' values
    - Calculating the difference
    - Applying this value to the dataframe
* Basically... 
    - df.rolling(window=7) returns a function. 
    - You can pass that function to a dataframe using lambda the same way you would any function

In [105]:
df.rolling(window=7).max().max()

Value    80.0
dtype: float64

In [107]:
df.rolling(window=7).min().min()

Value    10.0
dtype: float64

## Exercise

1. Using the previously loaded `dji.csv` data, up-sample the data to include the missing dates. Select a suitable fill value.
2. Calculating a moving average using the `rolling` function over a suitable period.
3. Down-sample the data by taking the monthly average and then calculate the month over month change.

#### 1. Using the previously loaded `dji.csv` data, up-sample (aka increase the frequency) the data to include the missing dates. Select a suitable fill value.

In [110]:
dji = pd.read_csv(
    filepath_or_buffer='dji.csv',
    parse_dates=['Date'],
    index_col='Date'
)
dji.head(5)

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-12-31,17987....,18043....,17820....,17823....,828400...,17823....
2014-12-30,18035....,18035....,17959....,17983....,474900...,17983....
2014-12-29,18046....,18073....,18021....,18038....,538700...,18038....
2014-12-26,18038....,18103....,18038....,18053....,525700...,18053....
2014-12-24,18035....,18086....,18027....,18030....,428700...,18030....


In [119]:
dji_fill = dji.resample('D').ffill()
dji_fill.head(5)

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-01-02,16572....,16573....,16416....,16441....,809600...,16441....
2014-01-03,16456....,16518....,16439....,16469....,727700...,16469....
2014-01-04,16456....,16518....,16439....,16469....,727700...,16469....
2014-01-05,16456....,16518....,16439....,16469....,727700...,16469....
2014-01-06,16474....,16532....,16405....,16425....,893800...,16425....


#### 2. Calculating a moving average using the `rolling` function over a suitable period.

In [121]:
dji_fill.rolling(window=7).mean().head(10)

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-05,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,16481....,16536....,16426....,16467....,8.1882...,16467....
2014-01-09,16467....,16529....,16421....,16467....,8.2315...,16467....
2014-01-10,16467....,16524....,16412....,16462....,8.4090...,16462....


#### 3. Down-sample (aka decrease the frequency) the data by taking the monthly average and then calculate the month over month change (aka the difference).

In [122]:
dji_fill_month = dji_fill.resample('m').mean()
dji_fill_month.head()

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-01-31,16316....,16368....,16212....,16268....,1.1102...,16268....
2014-02-28,15923....,16021....,15860....,15959....,2.1125...,15959....
2014-03-31,16295....,16390....,16235....,16303....,1.1461...,16303....
2014-04-30,16401....,16458....,16314....,16373....,1.0081...,16373....
2014-05-31,16555....,16607....,16503....,16568....,7.9961...,16568....


In [123]:
dji_fill_month.diff(1)

,DJI.Open,DJI.High,DJI.Low,DJI.Close,DJI.Volume,DJI.Adj.Close
Date,,,,,,
2014-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-28,-393.17...,-346.88...,-352.12...,-309.37...,1.0022...,-309.37...
2014-03-31,372.65...,368.48...,374.64...,344.14...,-9.6639...,344.14...
2014-04-30,105.89...,68.125658,79.333645,70.253743,-1.3791...,70.253743
2014-05-31,154.33...,149.26...,189.02...,194.59...,-2.0858...,194.59...
2014-06-30,276.14...,264.84...,283.63...,279.29...,1.9446...,279.29...
2014-07-31,164.51...,181.22...,142.77...,150.77...,-2.2862...,150.77...
2014-08-31,-244.92...,-234.43...,-233.51...,-227.66...,-1.4755...,-227.66...
2014-09-30,342.01...,341.18...,335.51...,335.58...,2.8914...,335.58...
